<a href="https://colab.research.google.com/github/sjnaj/DeepLearning/blob/master/Basic/ParaControl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.1631],
        [-0.0732]], grad_fn=<AddmmBackward0>)

In [ ]:
print(net[2].state_dict())
print(net.state_dict())


OrderedDict([('weight', tensor([[ 0.3301,  0.2680, -0.2128, -0.2718,  0.1404,  0.3295, -0.2350, -0.0581]])), ('bias', tensor([-0.1395]))])
OrderedDict([('0.weight', tensor([[ 0.4221, -0.2326, -0.2880, -0.4566],
        [-0.3731, -0.1875,  0.2557, -0.1890],
        [ 0.2729,  0.1038,  0.3698,  0.4719],
        [ 0.3306, -0.2816,  0.0531,  0.3940],
        [ 0.4455, -0.3500, -0.0747,  0.3772],
        [ 0.0829, -0.0651,  0.2804, -0.0888],
        [ 0.2771,  0.3105,  0.2880,  0.0805],
        [-0.2230, -0.3616, -0.4167, -0.2428]])), ('0.bias', tensor([-0.2929,  0.4570, -0.0948, -0.2618, -0.1257,  0.2878, -0.0461,  0.4120])), ('2.weight', tensor([[ 0.3301,  0.2680, -0.2128, -0.2718,  0.1404,  0.3295, -0.2350, -0.0581]])), ('2.bias', tensor([-0.1395]))])


In [ ]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
net[2].weight.grad == None#还未进行反向传播
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1395], requires_grad=True)
tensor([-0.1395])
('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [ ]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())#加入的module可以自定义名字
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.0067],
        [0.0067]], grad_fn=<AddmmBackward0>)

In [ ]:
print(rgnet)#打印整体结构

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [ ]:
rgnet[0][1][0].bias.data


tensor([-0.3234,  0.4344,  0.0976, -0.4692,  0.1774, -0.3487,  0.1939, -0.0219])

In [ ]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)#最后的下划线是inplace的意思
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0019,  0.0052,  0.0070, -0.0152]), tensor(0.))

In [ ]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [ ]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.1974,  0.3549, -0.5663, -0.4450])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [ ]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-9.2621, -6.1164, -5.5084,  0.0000],
        [ 8.2021, -8.6740,  0.0000,  0.0000]], grad_fn=<SliceBackward0>)

In [ ]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
